In [1]:
# Lab 5 Logistic Regression Classifier
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

torch.manual_seed(777)

In [2]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [3]:
x = np.loadtxt('./firstLastData.csv', delimiter=',', dtype=np.float32)
y = np.loadtxt('./firstLastResult.csv', delimiter=',', dtype=np.float32)
x_data = x
y_data = y

In [4]:
len(y_data[y_data==1])

8529

In [5]:
X = Variable(torch.from_numpy(x_data))
Y = Variable(torch.from_numpy(y_data))

In [6]:
X.size()

torch.Size([26039, 259])

In [7]:
# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))

#linear = torch.nn.Linear(259, 1, bias=True)
#sigmoid = torch.nn.Sigmoid()
#model = torch.nn.Sequential(linear, sigmoid)




In [8]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = torch.nn.Linear(259, 128, bias=True)
        self.linear2 = torch.nn.Linear(128, 64, bias=True)
        self.linear3 = torch.nn.Linear( 64, 1, bias= True)
        self.linearTemp = torch.nn.Linear(128,1, bias=True)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)

    
    def forward(self,x):
        x1 = self.bn1(F.relu(self.linear1(x)))
        x2= self.bn2(F.relu(self.linear2(x1)))
        x3 = F.relu( self.linear3(x2))
        return x3

In [9]:
dataset = TensorDataset(X, Y)

In [10]:
dataloader = DataLoader(dataset, batch_size=1000,
                        shuffle=False, num_workers=4)  

In [11]:
#train_loader = torch.utils.data.DataLoader( dataset=X, 
 #                                          batch_size=batch_size, 
 #                                          shuffle=False))

In [12]:
model = Model()

In [13]:
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = torch.optim.ASGD(model.parameters(), lr=0.01)
#criterion = torch.nn.CrossMapLRN2d(259)
#loss_fn = torch.nn.MSELoss(reduction='sum')
#criterion = torch.nn.CrossEntropyLoss()
loss = torch.nn.MSELoss()

y = Y
x = X

for step in range(31):

        # i = 0
        #for x in X:
       # y = Y[i]
        #i += 1
        #hypothesis = model(x)
        #model = Model()
        hypothesis = model(x)
        # cost/loss function
        # cost = -(y * torch.log(hypothesis) + (1 - y) * torch.log(1 - hypothesis)).mean()
        # cost = criterion( hypothesis, y )
        yy = torch.unsqueeze(y, 1)
        cost = loss( hypothesis, yy )
        #print( x )
        #print(index, cost.data.numpy() )
        
        # cost = criterion( hypothesis, Y.long().view(-1))
        # cons = loss_fn( hypothesis, Y)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print(step, cost.data.numpy())

In [14]:
#y = Y
#x = X

for step in range(10001):

    #i = 0
    #for x in X:
    #    y = Y[i]
    #    i += 1
    for index, (x, y) in enumerate(dataloader):    
        #hypothesis = model(x)
        #model = Model()
        hypothesis = model(x)
        # cost/loss function
        # cost = -(y * torch.log(hypothesis) + (1 - y) * torch.log(1 - hypothesis)).mean()
        # cost = criterion( hypothesis, y )
        y = torch.unsqueeze(y, 1)
        cost = loss( hypothesis, y )
        #print( x )
        #print(index, cost.data.numpy() )
        
        # cost = criterion( hypothesis, Y.long().view(-1))
        # cons = loss_fn( hypothesis, Y)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    if step % 100 == 0:
        print(step, cost.data.numpy())

0 0.28768688
100 0.19873802
200 0.16411231
300 0.13002186
400 0.1174947
500 0.10960496
600 0.1248325
700 0.11347391
800 0.104390614
900 0.10526061
1000 0.101416156
1100 0.097894296
1200 0.09401959
1300 0.09624871
1400 0.08970643
1500 0.0929377
1600 0.10360482
1700 0.09206671
1800 0.09286627
1900 0.10132153
2000 0.10501741
2100 0.08789153
2200 0.10015131
2300 0.0948515
2400 0.08779708
2500 0.10886282
2600 0.095691994
2700 0.10134592
2800 0.08723983
2900 0.08027524
3000 0.07629865
3100 0.073498204
3200 0.073326565
3300 0.07461673
3400 0.076863565
3500 0.1236442
3600 0.097790256
3700 0.08881593
3800 0.08276118
3900 0.08091621
4000 0.07433679
4100 0.074471585
4200 0.08007582
4300 0.07231541
4400 0.07065529
4500 0.08735709
4600 0.08018175
4700 0.07769222
4800 0.077655055
4900 0.07118963
5000 0.0830902
5100 0.08924157
5200 0.06659056
5300 0.06700837
5400 0.067552105
5500 0.064473286
5600 0.07181462
5700 0.10842196
5800 0.063816115
5900 0.06271102
6000 0.05986577
6100 0.064059295
6200 0.05923

In [15]:
# Accuracy computation
predicted = (model(X).data >0.5).float()
accuracy = (predicted == Y.data).float().mean()
print("\nHypothesis: ", hypothesis.data.numpy(), "\nCorrect (Y): ", predicted.numpy(), "\nAccuracy: ", accuracy)


Hypothesis:  [[0.09219705]
 [0.2429447 ]
 [0.11232521]
 [0.21641095]
 [0.04267703]
 [0.28946406]
 [0.18485974]
 [0.18347026]
 [0.        ]
 [0.        ]
 [1.0954914 ]
 [0.03813647]
 [0.0057293 ]
 [0.76806843]
 [0.13135074]
 [0.        ]
 [0.        ]
 [0.9079567 ]
 [0.        ]
 [0.        ]
 [0.11945467]
 [0.8994471 ]
 [0.05293113]
 [0.2585132 ]
 [0.2146074 ]
 [0.22515069]
 [1.0716106 ]
 [0.4463279 ]
 [0.5287185 ]
 [0.04352389]
 [0.51942587]
 [1.0315582 ]
 [0.84296453]
 [0.        ]
 [0.        ]
 [0.4643889 ]
 [0.24517463]
 [0.03122954]
 [1.1030728 ]] 
Correct (Y):  [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]] 
Accuracy:  tensor(0.6558)


In [16]:
model(X).data < 0.2

tensor([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]], dtype=torch.uint8)

In [17]:
Y

tensor([0., 0., 0.,  ..., 0., 0., 1.])

In [18]:
len(X)

26039

In [19]:
sum(predicted == Y.data)

tensor([203, 203, 203,  ..., 203, 203, 236], dtype=torch.uint8)

In [20]:
Y.data

tensor([0., 0., 0.,  ..., 0., 0., 1.])

In [21]:
len(predicted)

26039

In [22]:
y = torch.unsqueeze(Y,1)

In [23]:
sum(predicted==y.data)

tensor([14], dtype=torch.uint8)

In [24]:
sum(y.data)

tensor([8529.])

In [25]:
sum(y)

tensor([8529.])

In [26]:
sum(predicted)

tensor([1260.])

In [27]:
sum(y == predicted)

tensor([14], dtype=torch.uint8)

In [28]:
a = (y==predicted)

In [29]:
a

tensor([[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [0]], dtype=torch.uint8)

In [30]:
a.tolist()

[[1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1]